<a href="https://colab.research.google.com/github/yds04312/nownow/blob/master/final_use_yolo_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

훈련된 가중치와 cfg, name 파일을 가지고
실제로 웹 어플리케이션에 적용해보기 위해 값 추출하기!

https://bong-sik.tistory.com/16 님 코드를 참고하였습니다! 

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
print(cv2.__version__)

4.1.2


In [ ]:
%cd /content/drive/MyDrive/use

/content/drive/MyDrive/use


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Yolo 로드
net = cv2.dnn.readNet("confirm_yolov3_custom_train_2000.weights", "yolov3_custom_train.cfg")
classes = []
with open("yolo.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# 이미지 가져오기
img = cv2.imread("test_chin_9.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (608, 608), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)


# 정보를 화면에 표시
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.7:
            # Object detected
            print(confidence)
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
                   
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.4)


font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()
